In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
#---------------------(Read Dataset)-------------------------------------------
df=pd.read_csv("C:/Users/Asus/Desktop/Forecasting-covid-cases-MLDL/dataset-fr.csv")
df['date']=pd.to_datetime(df['date']).dt.date
df.set_index('date',inplace=True)
#------------------(Split Dataset)---------------------------------------------
TRAIN_SPLIT=int(len(df)*0.8)
#print(TRAIN_SPLIT)
lag=21 #for active cases (conf_j1)
#------------------(convert to supervised)-------------------------------------
#------------------(Multivariate_data function)--------------------------------
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)
#------------------------(set variable and convert to supervised)--------------
#the function just accept the array type
df=np.array(df)
#----------------------------(Scale Data)--------------------------------------
scaler =MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(df)
past_history =lag
#number of step for prediction
future_target = 1
STEP = 1
tscv = TimeSeriesSplit(n_splits = 3 )
x_train_multi, y_train_multi= multivariate_data(df, df[:,12], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
x_test_multi, y_test_multi = multivariate_data(df, df[:,12],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=False)

x_train_2d=x_train_multi.reshape(x_train_multi.shape[0],(x_train_multi.shape[1]*x_train_multi.shape[2]))
x_test_2d=x_test_multi.reshape(x_test_multi.shape[0],-1)                                
y_train_1d=y_train_multi.ravel()
y_test_1d=y_test_multi.ravel()
#------------------------------------(Random Forest)---------------------------
#Building the random forest model
RF_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [20,50,100,200,1000]
}
RF = RandomForestRegressor()
#cv=the number of folds to use for cross validation
RF_search = RandomizedSearchCV(RF,RF_grid,cv=tscv)
RF_search.fit(x_train_2d,y_train_1d)
RF_confirmed = RF_search.best_estimator_    
RF_confirmed.fit(x_train_2d,y_train_1d)

RandomForestRegressor(max_depth=100, min_samples_leaf=4, min_samples_split=5,
                      n_estimators=50)